# Linear Regression

### Imports

In [1]:
# imports
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as opt

from google.colab import drive
drive.mount('/content/drive')

# Seed 고정
torch.manual_seed(1)

Mounted at /content/drive


### 1. Data loading

|x|y|
|:--:|:--:|
|1|1|
|2|2|
|3|3|

In [2]:
# Data 수동으로 입력하기
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

print(x_train)
print(x_train.shape)
print(y_train)
print(y_train.shape)

tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])
tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])


### 2. Hypothesis (가정) and Cost

$$ y_{hypo} = H(x) = w ⋅ x + b $$

- Linear(선형)임을 가정한 경우 위 수식을 만족해야 한다.
- data가 3개인 경우,
  $$
	\begin{bmatrix} 
	y_1 \\
  y_2 \\
  y_3 \\
	\end{bmatrix}
  = w ⋅
  \begin{bmatrix} 
  x_1 \\
  x_2 \\
  x_3 \\
	\end{bmatrix}
  + b
	$$
  + 이를 만족하는 $w$와 $b$를 구해야 한다.
- Cost function으로 Mean-Square-Error를 이용한다.
  $$ \cfrac{1}{N} \sum_{n=1}^{N} = (y_{hypo,(n)} - y_{train,(n)})^2 $$

In [3]:
# Model 초기화 (입력 dim, 출력 dim)
model = nn.Linear(1, 1)

# Hypothesis 정의
y_hypo = model(x_train)
print(list(model.parameters()))
print(y_hypo)

# Cost(Mean Square Error) 계산
cost = func.mse_loss(y_hypo, y_train)
print(cost)

[Parameter containing:
tensor([[0.5153]], requires_grad=True), Parameter containing:
tensor([-0.4414], requires_grad=True)]
tensor([[0.0739],
        [0.5891],
        [1.1044]], grad_fn=<AddmmBackward0>)
tensor(2.1471, grad_fn=<MseLossBackward0>)


### 3. Cost Optimization
- Optimizer로 stochastic gradient descent를 이용한다

In [4]:
# Optimizer 설정 (learning rate = 0.01로 설정)
optimizer = opt.SGD(model.parameters(), lr=0.01)

In [5]:
# 반복
for epoch in range(1000):

  # Cost 계산 / mse_loss(가정에 의한 값, 참값)
  y_hypo = model(x_train)
  cost = func.mse_loss(y_hypo, y_train)

  # Cost를 이용해 model update
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100번 마다 중간결과 출력
  if epoch % 100 == 99:
    params = list(model.parameters())
    w = params[0].item()
    b = params[1].item()
    print('Epoch {:4d}/{} w: {:.3f} b: {:.3f} Cost: {:.6f}'.format(epoch+1, 1000, w, b, cost.item()))

Epoch  100/1000 w: 1.066 b: -0.150 Cost: 0.003255
Epoch  200/1000 w: 1.052 b: -0.118 Cost: 0.002011
Epoch  300/1000 w: 1.041 b: -0.093 Cost: 0.001243
Epoch  400/1000 w: 1.032 b: -0.073 Cost: 0.000768
Epoch  500/1000 w: 1.025 b: -0.057 Cost: 0.000475
Epoch  600/1000 w: 1.020 b: -0.045 Cost: 0.000293
Epoch  700/1000 w: 1.016 b: -0.035 Cost: 0.000181
Epoch  800/1000 w: 1.012 b: -0.028 Cost: 0.000112
Epoch  900/1000 w: 1.010 b: -0.022 Cost: 0.000069
Epoch 1000/1000 w: 1.008 b: -0.017 Cost: 0.000043


### 4. Multivariable Linear Regression

$$ y_{hypo} = H(x) = W ⋅ x + b $$

- Linear(선형)임을 가정한 경우 위 수식을 만족해야 한다

$$ y_{hypo} =
  \begin{bmatrix} 
	w_1 && w_2 && w_3 \\
	\end{bmatrix}
  ⋅
  \begin{bmatrix} 
	x_1 \\
  x_2 \\
  x_3 \\
	\end{bmatrix}
  +
  \begin{bmatrix} 
	b \\
	\end{bmatrix}
  $$

  + MSE cost를 최소화하는 $w$와 $b$를 구해야 한다

In [8]:
# Numpy를 이용해 csv file을 ndarray로 가져오기
dataset = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/data_linear_regression.csv', delimiter=',', dtype=np.float32)

# 순서를 random으로 섞기
np.random.shuffle(dataset)

# torch tensor로 변환
x_train = torch.FloatTensor(dataset[:,:-1])
y_train = torch.FloatTensor(dataset[:,[-1]])

# Linear model
model = nn.Linear(3, 1)

# Optimizer 설정
optimizer = opt.SGD(model.parameters(), lr=0.00001)

# 반복
for epoch in range(1000):
  y_hypo = model(x_train)
  cost = func.mse_loss(y_hypo, y_train)

  # model update
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100번 마다 중간결과 출력
  if epoch % 100 == 99:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch+1, 1000, cost.item()))

print(y_hypo - y_train)

Epoch  100/1000 Cost: 27.945921
Epoch  200/1000 Cost: 26.145187
Epoch  300/1000 Cost: 24.495098
Epoch  400/1000 Cost: 22.982988
Epoch  500/1000 Cost: 21.597193
Epoch  600/1000 Cost: 20.327085
Epoch  700/1000 Cost: 19.162960
Epoch  800/1000 Cost: 18.095854
Epoch  900/1000 Cost: 17.117683
Epoch 1000/1000 Cost: 16.220892
tensor([[-3.7555],
        [ 1.7373],
        [ 2.5786],
        [ 1.0688],
        [ 2.0861],
        [-2.3550],
        [-0.7313],
        [-8.5924],
        [-1.6001],
        [-5.1900],
        [-4.2887],
        [ 5.1373],
        [ 8.3312],
        [-2.7923],
        [-0.5103],
        [ 5.7041],
        [ 2.7283],
        [-6.4294],
        [-4.5285],
        [ 3.1929],
        [ 3.0094],
        [-0.2428],
        [ 2.9134],
        [-1.0264],
        [ 3.7314]], grad_fn=<SubBackward0>)


### 5. Assignment
- 행렬 방정식 풀기
  + 다음 행렬 방정식을 'Linear Regression'을 이용해 풀어보자
    * 적당한 learning rate를 찾아 1000 epoch 정도 계산해본다
    * 'Pseudo Inverse'를 이용한 풀이와 비교해본다
  + Hint : $y = wx$ 꼴로 변환해본다
    * $Ax = b$에서는 x가 미지수이지만, $y = wx$에서는 w가 미지수임에 주의!
    * linear model에서 b를 없애기 위해서 nn.Linear() 사용법을 검색해보자

In [33]:
# Seed 고정 (Linear Regression이 잘 사용이 되었는지 확인하기 위해 (랜덤성을 줄이기 위해)시드를 고정 시킨다.)
torch.manual_seed(1)

# Data 입력
a_train = torch.FloatTensor([[0,1],[1,1],[2,1],[3,1]])       # Tensor A 정의
b_train = torch.FloatTensor([[-1],[0.2],[0.9],[2.1]])        # Tensor B 정의
print(a_train)
print(b_train)

# Model 초기화 (입력 dim, 출력 dim)
model = nn.Linear(2, 1, False)    # False -> bias 없애기
print(list(model.parameters()))
print(model.parameters())

# Optimizer 설정 (learning rate = 0.01로 설정) , Stochastic gradient descent 모델을 사용하며 Multivariable function 이므로 Learning rate를 0.013으로 한 개의 입력일 때 보다 낮추어 설정한다.
optimizer = opt.SGD(model.parameters(), lr=0.01)


# 반복
for epoch in range(1000):                            

  # Cost 계산 / mse_loss (가정에 의한 값, 참값)
  b_hypo = model(a_train)                             # y_hypo 정의
  cost = func.mse_loss(b_hypo, b_train)               # cost 계산

  # cost를 이용해 model update
  optimizer.zero_grad()                               # parameter의 gradient 계산 시 0으로 초기화 해준다.
  cost.backward()                                     # 목표값과 측정값을 비교한 뒤 오차를 업데이트한다.
  optimizer.step()                                    # 계산한 gradient로 parameter를 업데이트한다.

  # 100번마다 중간결과 출력
  if epoch % 100 == 99:
    params = list(model.parameters())
    w = params[0][0][0].item()                              
    b = params[0][0][1].item()

    print('Epoch {:4d}/{} w: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
        epoch+1, 1000, w, b, cost.item()))

print(b_train)
print(b_hypo)

print(b_hypo - b_train)

tensor([[0., 1.],
        [1., 1.],
        [2., 1.],
        [3., 1.]])
tensor([[-1.0000],
        [ 0.2000],
        [ 0.9000],
        [ 2.1000]])
[Parameter containing:
tensor([[ 0.3643, -0.3121]], requires_grad=True)]
<generator object Module.parameters at 0x7ff4ba045f50>
Epoch  100/1000 w: 0.802 b: -0.528 Cost: 0.078024
Epoch  200/1000 w: 0.891 b: -0.717 Cost: 0.032368
Epoch  300/1000 w: 0.940 b: -0.822 Cost: 0.018524
Epoch  400/1000 w: 0.967 b: -0.879 Cost: 0.014327
Epoch  500/1000 w: 0.982 b: -0.911 Cost: 0.013054
Epoch  600/1000 w: 0.990 b: -0.929 Cost: 0.012668
Epoch  700/1000 w: 0.994 b: -0.938 Cost: 0.012551
Epoch  800/1000 w: 0.997 b: -0.944 Cost: 0.012515
Epoch  900/1000 w: 0.998 b: -0.946 Cost: 0.012505
Epoch 1000/1000 w: 0.999 b: -0.948 Cost: 0.012501
tensor([[-1.0000],
        [ 0.2000],
        [ 0.9000],
        [ 2.1000]])
tensor([[-0.9480],
        [ 0.0511],
        [ 1.0501],
        [ 2.0492]], grad_fn=<MmBackward0>)
tensor([[ 0.0520],
        [-0.1489],
       